In [1]:
import pandas as pd
import os

In [2]:
#get all the files in the forecast folder..........
def fileList(path):
    listOfFiles = os.listdir(path)
    files=[]
    
    for file in listOfFiles:
        fullPath = os.path.join(path, file) #full path for the file
        if os.path.isdir(fullPath): #if there are sub folders.....
            files = files + fileList(fullPath) #recursively loop through the sub folder
        else:
            files.append(fullPath)
                
    return files

all_the_files=fileList(r'E:\Desktop\forcast_july')

In [3]:
#all the xls files in forcast_july folder.......
for file in all_the_files:
    print(file)

E:\Desktop\forcast_july\july1\july1.xls
E:\Desktop\forcast_july\july10.xls
E:\Desktop\forcast_july\july11.xls
E:\Desktop\forcast_july\july12.xls
E:\Desktop\forcast_july\july13.xls
E:\Desktop\forcast_july\july14.xls
E:\Desktop\forcast_july\july15.xls
E:\Desktop\forcast_july\july16.xls
E:\Desktop\forcast_july\july17.xls
E:\Desktop\forcast_july\july18.xls
E:\Desktop\forcast_july\july19.xls
E:\Desktop\forcast_july\july2\july2.xls
E:\Desktop\forcast_july\july20.xls
E:\Desktop\forcast_july\july21.xls
E:\Desktop\forcast_july\july22.xls
E:\Desktop\forcast_july\july23.xls
E:\Desktop\forcast_july\july24.xls
E:\Desktop\forcast_july\july25.xls
E:\Desktop\forcast_july\july26.xls
E:\Desktop\forcast_july\july27.xls
E:\Desktop\forcast_july\july28.xls
E:\Desktop\forcast_july\july29.xls
E:\Desktop\forcast_july\july3.xls
E:\Desktop\forcast_july\july4.xls
E:\Desktop\forcast_july\july5.xls
E:\Desktop\forcast_july\july6.xls
E:\Desktop\forcast_july\july7.xls
E:\Desktop\forcast_july\july8.xls
E:\Desktop\forca

In [4]:
##############
data_frames=[]
for file in all_the_files:
    df=pd.read_excel(file)
    df.columns = df.iloc[0] #make columns as 0th row items..
    columns=df.columns
    date=columns[1].date() 
    df=df.drop(0) #drop 0th row items
    df.columns = df.iloc[0] #make columns as 0th row items..
    df=df.drop(1)  #drop 1st row items
    plants_temp=df.iloc[:,0] #all the items in plant column(this includes unnessasary items)
    plants=[]
    #get only the correct plants...
    for plant in plants_temp:
        if plant=='Total Generation':
            break
        else:
            plants.append(plant)
    df = df.loc[:, df.columns.notnull()] #remove nan columns.............
    z=len(plants) #only get the relevant rows...............
    df=df.head(z) #only get the dataframe which contains the needed rows...
    df.insert(0, 'Power_Plant', plants)
    df=df.drop(columns=df.columns[-1:],axis=1) #drop the last column.......
    df=pd.melt(df,id_vars=['Power_Plant']) #reshape the df using melt function...
    df['Date']=date
    data_frames.append(df)

In [5]:
#merge all the dataframes of all the days...
from functools import reduce
df_merged = reduce(lambda  left,right: pd.merge(left,right,how='outer'), data_frames)
df_merged.shape

(62352, 4)

In [6]:
df_merged.head(2000)

,Power_Plant,1,value,Date
0,WPS,00:00:00,0.0,2020-07-01
1,Cany,00:00:00,0.0,2020-07-01
2,Olax,00:00:00,10.0,2020-07-01
3,Nlax,00:00:00,60.636,2020-07-01
4,Polp,00:00:00,78.479,2020-07-01
...,...,...,...,...
1995,PCCP II,22:30:00,270.0,2020-07-01
1996,PCCP III,22:30:00,270.0,2020-07-01
1997,KPS,22:30:00,0.0,2020-07-01
1998,CEB_EMERGENCY,22:30:00,0.0,2020-07-01


In [10]:
df_merged=df_merged.rename(columns = {'Power_Plant':'Plant',1:'Time','value':'Power'}) #rename columns...

In [11]:
df_merged.head()

,Plant,Time,Power,Date
0,WPS,00:00:00,0.0,2020-07-01
1,Cany,00:00:00,0.0,2020-07-01
2,Olax,00:00:00,10.0,2020-07-01
3,Nlax,00:00:00,60.636,2020-07-01
4,Polp,00:00:00,78.479,2020-07-01
